In [192]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [193]:
data = pd.read_csv('./data/train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73995 entries, 0 to 73994
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         73995 non-null  uint64 
 1   org_id                          73995 non-null  uint64 
 2   rating                          73995 non-null  float64
 3   ts                              73995 non-null  int64  
 4   user_city                       73995 non-null  object 
 5   org_city                        73995 non-null  object 
 6   average_bill                    45433 non-null  float64
 7   rating_org                      73995 non-null  float64
 8   rubrics                         73995 non-null  object 
 9   food_delivery                   73995 non-null  int64  
 10  breakfast                       73995 non-null  int64  
 11  takeaway                        73995 non-null  int64  
 12  summer_terrace                  

чистим выбросы

In [194]:
data1 = data.copy()

std = data1.rating_org.std()
mean = data1.rating_org.mean()
data1 = data1[(data1.rating_org < mean + 3 * std) & (data1.rating_org > mean - 3 * std)]

data1.describe()

,user_id,org_id,rating,ts,average_bill,rating_org,food_delivery,breakfast,takeaway,summer_terrace,...,call_button,wheelchair_accessible,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods
count,7.297900e+04,7.297900e+04,72979.000000,72979.000000,44888.000000,72979.000000,72979.000000,72979.000000,72979.000000,72979.000000,...,72979.000000,72979.000000,72979.000000,72979.000000,72979.000000,72979.000000,72979.000000,72979.000000,72979.000000,72979.000000
mean,9.199156e+18,9.037064e+18,4.340550,714.279834,1078.317145,4.338128,0.758328,0.579728,0.653010,0.603722,...,0.008126,0.007482,0.017882,0.002179,0.002453,0.000959,0.000945,0.001247,0.002973,0.001220
std,5.312869e+18,5.274596e+18,1.110908,293.254623,2709.470983,0.259649,0.428100,0.493606,0.476016,0.489127,...,0.089776,0.086173,0.132523,0.046626,0.049465,0.030956,0.030734,0.035290,0.054449,0.034901
min,1.445171e+14,1.626686e+15,1.000000,0.000000,500.000000,3.395062,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.621896e+18,4.504716e+18,4.000000,505.000000,500.000000,4.190476,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.193479e+18,9.104453e+18,5.000000,698.000000,1000.000000,4.372516,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.376540e+19,1.348790e+19,5.000000,958.000000,1000.000000,4.526316,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.844666e+19,1.844531e+19,5.000000,1216.000000,246500.000000,5.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [195]:
data = data1

In [196]:
D = [] # на сколько rating_mean отклоняется от rating_org?

for oid in data.org_id.unique():
    d = data[data.org_id == oid]
    m = d.rating.mean()
    D.append(d.rating_org.iloc[0] - m)
    data.loc[d.index, ["rating_mean"]] = m

np.mean(D), np.std(D)

(-0.004244497798535122, 0.9214592079726398)

норм.

In [197]:
from sklearn.neighbors import KNeighborsRegressor

X = data.rating_mean.to_numpy().reshape(-1, 1)
Y = data.rating_org.to_numpy()

In [205]:
model = KNeighborsRegressor(n_neighbors=1000)
model.fit(X, Y)
model.score(X, Y)

0.27346078633354465

Предсказываем рейтинг

In [199]:
final = pd.read_csv('./data/test_x.csv')

In [200]:
final.rating.describe()

count    24665.000000
mean         4.321589
std          1.128989
min          1.000000
25%          4.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: rating, dtype: float64

In [201]:
for oid in final.org_id.unique():
    d2 = final[final.org_id == oid]
    m = d2.rating.mean()
    final.loc[d2.index, ["rating_mean"]] = m

In [202]:
results = pd.DataFrame({
    'rating_org': model.predict(final.rating_mean.to_numpy().reshape(-1, 1)),
})

In [203]:
results

,rating_org
0,4.284975
1,4.214571
2,4.325247
3,4.383325
4,4.483976
...,...
24660,4.325247
24661,4.325247
24662,4.153643
24663,4.383325


In [204]:
results.to_csv("./solution2.csv", index_label='id')